# **DATA PROJECT 2024: DO INTERENATIONAL WORKERS IN DENMARK REMEDY LABOR SHORTAGE?**

By Emma Knippel, Anna Abildsjov and Oscar Nyholm

# Table of contents
* [Setup](#toc0_)   

* [Introduction](#toc1_) 

* [Read and clean data](#toc2_)    

* [Exploring the data sets](#toc3_)    

* [Merging data sets of employment and international labor](#toc4_)   

* [Analysis](#toc5_) 

* [Conclusion](#toc6_) 

## <a id='toc0_'></a>[Setup](#toc0_)

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib_venn import venn2
import json

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# installing API reader, that will allow to load data from DST.
%pip install git+https://github.com/alemartinello/dstapi
%pip install pandas-datareader

import pandas_datareader # install with `pip install pandas-datareader`
from dstapi import DstApi # install with `pip install git+https://github.com/alemartinello/dstapi`

import dataproject #importing our own py-file with our code.

  Cloning https://github.com/alemartinello/dstapi to /private/var/folders/24/czmv85dj1_3dcc2tc4x8kd0r0000gn/T/pip-req-build-2dcypd8b
  Running command git clone --quiet https://github.com/alemartinello/dstapi /private/var/folders/24/czmv85dj1_3dcc2tc4x8kd0r0000gn/T/pip-req-build-2dcypd8b
  Resolved https://github.com/alemartinello/dstapi to commit d9eeb5a82cbc70b7d63b2ff44d92632fd77123a4
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## <a id='toc1_'></a>[Introduction](#toc1_)

## <a id='toc1_'></a>[Read and clean data](#toc1_)

In [ ]:
# importing the actual data from DST
employees = DstApi('LBESK03')
lb_short_service = DstApi('KBS2')
lb_short_manu = DstApi('BARO3')
lb_short_cons = DstApi('KBYG33')
with open('International Labor.json', 'r') as f:
    int_data = json.load(f)
int_lb = pd.DataFrame(int_data)

In [2]:
tabsum = employees.tablesummary(language = 'en')
display(tabsum)
for variable in tabsum['variable name']:
    print(variable+':')
    display(employees.variable_levels(variable, language='en'))

NameError: name 'employees' is not defined

In [ ]:
tabsum2 = lb_short_service.tablesummary(language = 'en')
display(tabsum2)
for variable in tabsum2['variable name']:
    print(variable+':')
    display(lb_short_service.variable_levels(variable, language='en'))

In [ ]:
tabsum3 = lb_short_manu.tablesummary(language = 'en')
display(tabsum3)
for variable in tabsum3['variable name']:
    print(variable+':')
    display(lb_short_manu.variable_levels(variable, language='en'))

In [ ]:
tabsum4 = lb_short_cons.tablesummary(language = 'en')
display(tabsum4)
for variable in tabsum4['variable name']:
    print(variable+':')
    display(lb_short_cons.variable_levels(variable, language='en'))

In [ ]:
params = employees._define_base_params(language='en')

# For the employment data, we first define our parameters so that we get only data from january 2014 to january 2024.
params = {'table': 'LBESK03',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'BRANCHEDB071038', 'values': ['*']},
  {'code': 'Tid', 'values': ['>2013M12<=2024M01']}]}

# Then, we retract the data we defined, drop the column of industry since we do not need it and rename the columns to english, simple titles.
empl = employees.get_data(params=params)
empl.drop(['BRANCHEDB071038'], axis=1, inplace=True)
empl.rename(columns = {'INDHOLD':'employees', 'TID':'time'}, inplace=True)
empl.head(5)

In [ ]:
params2 = lb_short_service._define_base_params(language='en')

# For the labor shortaage data, we need to sort through the dataset a bit more when defining out variables:
# We need to specify which industries we want to get data from, since the dataset contains both broad and narrow categories.
#Furhtermore, we want to get data only for the labor shortage and from january 2014 to january 2024.
params2 = {'table': 'KBS2',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'BRANCHE07', 'values': [
     '000',
     '005',
     '015',
     '035',
     '045',
     '060',
     '065',
     '080'
 ]},
  {'code': 'TYPE', 'values': ['MAAK']},
  {'code': 'Tid', 'values': ['>2013M12<=2024M01']}]}

# We retrieve the parameters and sort the data by time and industry.
lab_short_service = lb_short_service.get_data(params=params2)
lab_short_service.sort_values(by = ['TID', 'BRANCHE07'], inplace=True)

# Then, we drop the column 'TYPE', since we only have data for the labor shortage anyways, and this column would otherwise be used to split the data into diffeereeent categories of production limitations.
# We rename the columns to english, simple titles, and the rows to the industry names rather than the industry codes.
# We also drop the old index and reset it.
lab_short_service.drop(['TYPE'], axis = 1, inplace = True)
lab_short_service.rename(columns = {'BRANCHE07':'industry', 'TID':'Time', 'INDHOLD':'Labor Shortage'}, inplace=True)
lab_short_service.reset_index(drop=True, inplace=True)
lab_short_service.head(10)

In [ ]:
lab_short_service['industry'] = lab_short_service.industry.replace({
    '000':'Total','SERVICES TOTAL':'Total',
    '005':'Transport','TRANSPORT (49-53)':'Transport',
    '015':'Hotels and restaurants','TOURISME (55-56, 79)': 'Hotels and restaurants',
    '035':'Information and communication','COMMUNICATION AND INFORMATION (58, 61-63)':'Information and communication',
    '045':'Finance and real estate','FINANCE, INSURANCE AND REAL ESTATE (64-65, 68)': 'Finance and real estate',
    '060':'Research and consultancy','CONSULTANCY, RESEARCH AND OTHERS (69-74)':'Research and consultancy',
    '065':'Cleaning etc.', 'CLEANING AND OTHER OPERATIONEL SERVICE (77-78, 81-82)':'Cleaning etc.',
    '080':'Culture and leisure','ARTS, RECREATION AND OTHER SERVICES (90-95)':'Culture and leisure',
    })

lab_short_service.head(100)


In [ ]:
params3 = lb_short_manu._define_base_params(language='en')

params3 = {'table': 'BARO3',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'BRANCHE07', 'values': ['C']},
  {'code': 'TYPE', 'values': ['AMA']},
  {'code': 'Tid', 'values': ['>2013K4<=2024K1']}]}

lab_short_manu = lb_short_manu.get_data(params=params3)
lab_short_manu.sort_values(by = ['TID'], inplace=True)
lab_short_manu.rename(columns={'BRANCHE07': 'Industry', 'TID': 'Time', 'INDHOLD': 'Labor Shortage'}, inplace=True)
lab_short_manu.reset_index(drop=True, inplace=True)
lab_short_manu['Time'] = pd.to_datetime(lab_short_manu['Time'], format='mixed')
lab_short_manu.head(10)

In [ ]:
params4 = lb_short_cons._define_base_params(language='en')

params4 = {'table': 'KBYG33',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'BRANCHE07', 'values': ['F']},
  {'code': 'TYPE', 'values': ['AMA']},
  {'code': 'Tid', 'values': ['>2013M12<=2024M01']}]}

lab_short_cons = lb_short_cons.get_data(params=params4)
lab_short_cons.sort_values(by = ['TID'], inplace=True)
lab_short_cons.rename(columns={'BRANCHE07': 'Industry', 'TID': 'Time', 'INDHOLD': 'Labor Shortage'}, inplace=True)
lab_short_cons.reset_index(drop=True, inplace=True)
lab_short_cons['Time'] = pd.to_datetime(lab_short_cons['Time'], format='%YM%m')
lab_short_cons.head(5)

**Cleaning the data on International workers sorted from JobIndsats**

In [14]:
import dataproject
int_labor = dataproject.clean_json_data()
int_labor.head(5)

Before cleaning, the JSON datafile from JobIndsats contains 1089 observations and 5 variables.
We have removed two columns and renamed the remaining.
The dataset now contains 1089 observations and 3 variables.
All our observations are of type: <class 'str'>. We want them to be integers.
The observations are now of type: <class 'numpy.float64'> and the first observation is: 2.184
The observations are now of type: <class 'numpy.int64'> and the first observation is: 2184
We convert our time Variable into datetime variables.
We now convert the DataFrame using the .pivot method, using time as index, industries as columns and international labor as our observations.
All our industries are in Danish, so we rename them to English.
For our dataset to match the data from DST, we sum over all industries to get the total and combine four of the industires so that they match
Lastly, we drop the industries, that we have just combined to make new ones.
The cleaned dataset now contains 8 columns (indu

industry,hotels_restaurents,information_communictaion,cleaning_etc,transport,research_consultancy,total,finance_real_estate,culture_leisure_other
time,,,,,,,,
2014M01,17609,6657,26549,14777,9039,86158,4684,6843
2014M02,17957,6815,26792,14732,9137,86993,4690,6870
2014M03,18481,7001,27667,14975,9292,89550,4841,7293
2014M04,19209,7136,28547,15177,9611,92029,4928,7421
2014M05,19909,7342,29802,15380,9539,94728,5103,7653


## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

Statisk plot af:
- mangel på medarbejdere i de 3 sektorer, januar 2014 til januar 2024

In [ ]:
data = {'Time': ['2014M01', '2014M02', '2014M03']}
df = pd.DataFrame(data)

df[['Year', 'Month']] = df['Time'].str.extract(r'(\d{4})M(\d{2})')

df['Time'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1))

df.drop(['Year', 'Month'], axis=1, inplace=True)

lab_short_service['Time'] = pd.to_datetime(lab_short_service['Time'], format='%YM%m')
lab_short_manu['Time'] = pd.to_datetime(lab_short_manu['Time'], format='%Y-%m-%d')
lab_short_cons['Time'] = pd.to_datetime(lab_short_cons['Time'], format='%Y-%m-%d')

plt.plot(lab_short_service['Time'], lab_short_service['Labor Shortage'], label='Service sector')
plt.plot(lab_short_manu['Time'], lab_short_manu['Labor Shortage'], label='Manufacturing sector')
plt.plot(lab_short_cons['Time'], lab_short_cons['Labor Shortage'], label='Construction sector')

plt.xlabel('Time')
plt.ylabel('Labor Shortage')
plt.title('Labor Shortage across Sectors (January 2014 - January 2024)')
plt.legend()

plt.show()


**Interactive plot** :

Her skal vi have udviklingen i antal internationale medarbejdere i hver af servicebrancherne med drop-down

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.